In [15]:
!pip install transformers torch gradio --quiet


In [16]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import gradio as gr


In [17]:
# Using DistilBERT fine-tuned on SST-2 (binary sentiment)
MODEL_NAME = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)


In [18]:
def analyze_sentiment(text):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt")

    # Model output
    outputs = model(**inputs)

    # Apply softmax to convert logits → probabilities
    scores = F.softmax(outputs.logits, dim=1)

    # Extract positive/negative scores
    positive = float(scores[0][1])
    negative = float(scores[0][0])

    # Compound score: simple (pos - neg)
    compound = positive - negative

    # Label assignment
    label = "Positive" if compound >= 0 else "Negative"

    return label, compound


In [45]:
def generate_response(sentiment_label):
    if sentiment_label == "Positive":
        return "Nice!I'm glad to hear that. How can I assist you next?"
    else:
        return "I'm sorry you're feeling this way.Want to talk about it?"

In [46]:
EXIT_WORDS = ["bye", "goodbye", "see you", "stop", "exit"]
GREETINGS = ["hey", "hi", "hello", "good morning", "good evening", "good afternoon"]

def chatbot_controller(user_text):
    text = user_text.lower().strip()

    # Exit logic
    if any(word == text for word in EXIT_WORDS):
        return "Session ended. Thank you."

    # Greeting logic
    if any(word == text for word in GREETINGS):
        return "Hello. How can I help you today?"

    # Sentiment flow
    label, compound = analyze_sentiment(user_text)
    bot_reply = generate_response(label)

    return f"{bot_reply}\n\n(Sentiment: {label}, score={compound:.3f})"


In [47]:
def gradio_chat(user_message, history):
    response = chatbot_controller(user_message)
    history.append((user_message, response))
    return history, ""


In [48]:
with gr.Blocks(theme="soft") as demo:

    gr.Markdown(
        """
        <h1 style="text-align:center; color:#4A90E2;"> Transformer Sentiment Chatbot</h1>
        <p style="text-align:center; font-size:16px;">
            Talk to the chatbot below. Say <b>bye</b> to end the conversation.
        </p>
        <hr>
        """
    )

    chatbot_ui = gr.Chatbot(
        height=350,
        label="Chat Window",
        bubble_full_width=False,
    )

    user_input = gr.Textbox(
        label="Type your message...",
        placeholder="Write something here...",
    )

    send_btn = gr.Button("Send", variant="primary", size="lg")

    def gradio_chat(user_message, history):
        reply = chatbot_controller(user_message)

        # Add proper conversational format
        history.append(
            (f" {user_message}", f" {reply}")
        )
        return history, ""

    send_btn.click(
        gradio_chat,
        inputs=[user_input, chatbot_ui],
        outputs=[chatbot_ui, user_input]
    )


/tmp/ipython-input-1689523438.py:1: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme="soft") as demo:
/tmp/ipython-input-1689523438.py:13: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot(
/tmp/ipython-input-1689523438.py:13: DeprecationWarning: The 'bubble_full_width' parameter will be removed in Gradio 6.0. This parameter no longer has any effect.
  chatbot_ui = gr.Chatbot(
/tmp/ipython-input-1689523438.py:13: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if y

In [51]:
demo.launch()


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ec17cbc1fa7279b351.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
